## Random trading activity

Generate some random trading activity on OP Goerli testnet

* Create a set of accounts
* Fund those accounts
* Generate random trading activity

In [ ]:
import os
import time
import random
from synthetix import Synthetix
from dotenv import load_dotenv

load_dotenv()

In [ ]:
snx = Synthetix(
    provider_rpc=os.getenv('TESTNET_RPC'), # An OP Goerli RPC endpoint
    address=os.getenv('ADDRESS'), # A wallet address
    private_key=os.getenv('PRIVATE_KEY'), # A wallet private key
    network_id=420, # OP Goerli
)

In [ ]:
# print the available account IDs
snx.perps.account_ids

In [ ]:
# create a range of account_ids
account_ids = range(100, 110)

for account_id in account_ids:
    id_tx = snx.perps.create_account(account_id=account_id, submit=True)


In [ ]:
# put collateral in each account
for account_id in account_ids:
    modify_collateral = snx.perps.modify_collateral(50, market_id=0, account_id=account_id, submit=True)


In [ ]:
# check the collateral in each account
for account_id in account_ids:
    collateral = snx.perps.get_collateral_balances(account_id=account_id)
    print(f"Account {account_id}: {collateral}")

In [ ]:
# randomize a wait and then a new transaction
last_tx = {}

while True:
    order_size = random.uniform(0.01, 0.05)
    order_side = random.choice([-1, 1])
    market_name = random.choice(['ETH', 'BTC'])
    order_size = order_size * order_side
    wait_time = random.uniform(0.2, 1)

    account_id = random.choice(account_ids)
    if account_id in last_tx:
        if last_tx[account_id] > time.time() - 60:
            continue

    try:
        
        commit = snx.perps.commit_order(order_size, market_name=market_name, account_id=account_id,  submit=True)
    except Exception as e:
        print(f'Tx failed: {e}')
        continue

    last_tx['account_id'] = time.time()
    time.sleep(wait_time)


In [ ]:
# close all positions
for account_id in account_ids:
    for market_name in ['ETH', 'BTC']:
        open_position = snx.perps.get_open_position(account_id=account_id, market_name=market_name)
        if abs(open_position['position_size']) > 0:
            print(f'Closing position for {account_id} on {market_name}')
            try:
                close_position = snx.perps.commit_order(-open_position['position_size'], market_name=market_name, account_id=account_id, submit=True)
            except Exception as e:
                print(f'Tx failed: {e}')
                continue
